Extract the listed prerequisites from Carta tables

gmdavis, Feb 2020

In [190]:
import pandas as pd
import numpy as np
import ast
import pickle

In [191]:
REQS_PATH = "../data/requisites.txt"
PICKLE_PATH = "../data/official_reqs.pkl"

In [192]:
# wrangling table
reqs_df = pd.read_csv(REQS_PATH, sep = "\t")
reqs_df["course_name"] = reqs_df["SUBJECT"] + reqs_df["CATALOG_NBR"]
reqs_df.fillna("{}", inplace = True)
reqs_df = reqs_df[['course_name', 'requisites']][1:]
reqs_df

,course_name,requisites
1,AA100,"{""coreq"": [], ""prereq"": [""MATH 20"", ""MATH 21"",..."
2,AA102,{}
3,AA103,{}
4,AA108N,{}
5,AA109Q,{}
...,...,...
30958,WELLNESS290,{}
30959,WELLNESS291,{}
30960,WELLNESS293,"{""coreq"": [], ""prereq"": [], ""postreq"": [], ""re..."
30961,WELLNESS299,{}


In [193]:
reqs_df.dtypes

course_name    object
requisites     object
dtype: object

In [194]:
ast.literal_eval(reqs_df["requisites"][1])

{'coreq': [],
 'prereq': ['MATH 20', 'MATH 21', 'MATH 41', 'MATH 42'],
 'postreq': ['AA 206', 'AA 241A', 'AA 241B'],
 'recommend': []}

In [195]:
ast.literal_eval(reqs_df["requisites"][2])

{}

In [196]:
reqs_df["requisites"].apply(ast.literal_eval)

1        {'coreq': [], 'prereq': ['MATH 20', 'MATH 21',...
2                                                       {}
3                                                       {}
4                                                       {}
5                                                       {}
                               ...                        
30958                                                   {}
30959                                                   {}
30960    {'coreq': [], 'prereq': [], 'postreq': [], 're...
30961                                                   {}
30962    {'coreq': [], 'prereq': [], 'postreq': [], 're...
Name: requisites, Length: 30962, dtype: object

In [197]:
reqs_df

,course_name,requisites
1,AA100,"{""coreq"": [], ""prereq"": [""MATH 20"", ""MATH 21"",..."
2,AA102,{}
3,AA103,{}
4,AA108N,{}
5,AA109Q,{}
...,...,...
30958,WELLNESS290,{}
30959,WELLNESS291,{}
30960,WELLNESS293,"{""coreq"": [], ""prereq"": [], ""postreq"": [], ""re..."
30961,WELLNESS299,{}


In [198]:
test = ast.literal_eval(reqs_df["requisites"][1])
type(test)
test['prereq']

['MATH 20', 'MATH 21', 'MATH 41', 'MATH 42']

In [199]:
reqs_df["requisites"].apply(lambda x: dict(ast.literal_eval(x)))

1        {'coreq': [], 'prereq': ['MATH 20', 'MATH 21',...
2                                                       {}
3                                                       {}
4                                                       {}
5                                                       {}
                               ...                        
30958                                                   {}
30959                                                   {}
30960    {'coreq': [], 'prereq': [], 'postreq': [], 're...
30961                                                   {}
30962    {'coreq': [], 'prereq': [], 'postreq': [], 're...
Name: requisites, Length: 30962, dtype: object

In [200]:
type(ast.literal_eval(reqs_df["requisites"][1]))

dict

In [201]:
official_reqs = {}

# iterate through the rows, use ast.literal_eval on it and save to official reqs
for row in reqs_df.itertuples(index = False):
    course_name = row[0]
    reqs_pd = ast.literal_eval(row[1])
    reqs_list = []
    if len(reqs_pd) > 0:
        reqs_list += reqs_pd["coreq"]
        reqs_list += reqs_pd["prereq"]
        reqs_list += reqs_pd["recommend"]
        reqs_list = [req.replace(" ", "") for req in reqs_list]
    official_reqs[course_name] = reqs_list

In [202]:
official_reqs

{'AA100': ['MATH20', 'MATH21', 'MATH41', 'MATH42'],
 'AA102': [],
 'AA103': [],
 'AA108N': [],
 'AA109Q': [],
 'AA113N': [],
 'AA115N': [],
 'AA115Q': [],
 'AA116N': [],
 'AA116Q': [],
 'AA118N': [],
 'AA119N': [],
 'AA120Q': [],
 'AA121Q': [],
 'AA122N': [],
 'AA131': [],
 'AA135': [],
 'AA136A': [],
 'AA136B': [],
 'AA141': [],
 'AA146A': [],
 'AA146B': [],
 'AA149': [],
 'AA151': [],
 'AA156': [],
 'AA173': [],
 'AA174A': [],
 'AA174B': [],
 'AA190': [],
 'AA199': [],
 'AA200': ['AA210A'],
 'AA200A': [],
 'AA200B': [],
 'AA201A': [],
 'AA201B': ['AA201A'],
 'AA202': [],
 'AA203': [],
 'AA204': [],
 'AA208': ['AA210A'],
 'AA210A': [],
 'AA210B': ['AA210A'],
 'AA212': ['ENGR205', 'EE263'],
 'AA214': [],
 'AA214A': ['CME108'],
 'AA214B': ['CME206'],
 'AA214C': ['AA214B'],
 'AA215A': ['AA214B'],
 'AA215B': ['AA214B'],
 'AA218': [],
 'AA222': [],
 'AA228': [],
 'AA229': ['AA228', 'CS238', 'CS221'],
 'AA236A': [],
 'AA236B': ['AA236A'],
 'AA236C': [],
 'AA236D': [],
 'AA238': [],
 'AA240'

In [203]:
print(official_reqs["CS224U"])
print(official_reqs["CS221"])

['LINGUIST180', 'CS124', 'CS224N', 'CS224S', 'CS221', 'LINGUIST130A', 'LINGUIST130B', 'CS157', 'PHIL150']
['CS103', 'CS106B', 'CS106X', 'CS107', 'CS109']


In [205]:
with open(PICKLE_PATH, "wb") as file:
    pickle.dump(official_reqs, file)